# Kinematics

In [102]:
# import
import numpy as np
import math

In [103]:
def leg_FK(origin_bot, angles, links):
    """
    # Hexapod Leg Forward-Kinematics function
    # Author: Eirik Strandman
    # Version: V1.0

    # Description:
    A function that takes in robot origin, joint angles and leg configuration and 
    calculates the position of the joints for a two-member hexapod leg.


    # Inputs
    - origin_bot: (x,y,z)[mm] - The origin of the robot coordinate system.
    - angles: [J0, J1, J2, J3][mm] - The joint angles.
    - links: [L0, L1, L2, L3][mm] - The link lengths of the robot leg.  

    # Output
    - [[J1_pos, J2_pos, J3_pos, EF_pos](x,y,z)[mm] - Numpy array containing estimated joint positions.
    """


    # Find x-values
    x2 = links[0] + links[1]
    x3 = x2 + links[2]*np.cos(angles[2])
    x4 = x3 + links[3]*np.cos(angles[2] + angles[3])

    # Find z-values
    z2 = origin_bot[2] + links[2]*np.sin(angles[2])
    z3 = z2 + links[3]*np.sin(angles[2] + angles[3])

    # Find joint positions
    J1_pos = np.array([ links[0]*np.cos(angles[0]),  links[0]*np.sin(angles[0]),  origin_bot[2] ])
    J2_pos = np.array([ x2*np.sin(angles[0] + angles[1]), x2*np.cos(angles[0] + angles[1]), origin_bot[2] ])
    J3_pos = np.array([ x3*np.sin(angles[0] + angles[1]), x3*np.cos(angles[0] + angles[1]), z2 ])
    EF_pos = np.array([ x4*np.sin(angles[0] + angles[1]), x4*np.cos(angles[0] + angles[1]), z3 ])

    # Return leg position as 2D array.
    leg_pos = np.array([J1_pos, J2_pos, J3_pos, EF_pos])
    return leg_pos

In [104]:
def leg_IK(origin_bot, point, links, leg_angle):
    """
    # Hexapod Leg Inverse-Kinematics function
    # Author: Eirik Strandman
    # Version: V1.0

    # Description:
    A function that takes in the robot origin, leg configuration and desired 
    end-effector location and estimates joint angles for a 2 member hexapod length.


    # Inputs
    - origin_bot: (x,y,z)[mm] - The origin of the robot coordinate system.
    - point: (x,y,z)[mm] - Desired End-Effector (EF) location.
    - links: [L0, L1, L2, L3][mm] - The link lengths of the robot leg.
    - leg_angle: [rad] - The static default angle of the leg relative to the robot body.    

    # Output
    - [J0_ang, J1_ang, J2_ang, J3_ang][rad] - Numpy array containing estimated joint angles.
    """


    # Calculate leg origin relative to robot origin leg angle.
    origin_leg = np.array([
         (links[0]*np.cos(leg_angle) + origin_bot[0]), # x
         (links[0]*np.sin(leg_angle) + origin_bot[1]), # y
         (origin_bot[2]) ])                                # z

    # Translate to local leg coordinate system
    point_leg = point - origin_leg 
    print(point_leg)
 
    # Find joint angle for J1
    J1_ang = math.atan2(point_leg[0], point_leg[1]) # atan(x,y), note: x and y are reversed so x-axis points forward.

    # Find 2D resultant vector H for P_x and P_y with Link 1 subtracted.
    H = np.sqrt( (point_leg[0]**2) + (point_leg[1]**2) ) - links[1]

    # Find resultant vector L from H and Z. 
    L = np.sqrt( (H**2) + (point_leg[2]**2) )

    # Find joint angle J3
    J3_ang =  math.acos( ( (links[2]**2) + (links[3]**2) - (L**2) ) / (2 * links[2] * links[3]))
    #print(J3_ang)
    
    # Find joint angle J2
    B = math.acos( ( L**2 + (links[2]**2) - (links[3]**2) ) / ( 2 * L * links[2]))
    #print(B)
    A = math.atan2( point_leg[2], H )
    print("A")
    print(np.rad2deg(A))
    print("B")
    print(np.rad2deg(B))
    J2_ang = B + A

    # Apply offsets
    J0_ang = leg_angle
    J1_ang = J1_ang
    J2_ang = J2_ang 
    J3_ang = J3_ang - np.pi

    # Store as numpy array
    angles = np.array([J0_ang, J1_ang, J2_ang, J3_ang])
    return angles

In [105]:
# Configuration
links = np.array([  72.75,  54.0, 71.0, 73.5 ]) # Link lengths [mm])
#offset = np.array([0, 45,  45,  135 ])
#offset = np.deg2rad(offset)
offset = np.array([ 0,         0,  0,  0])
start = np.array([0, 0, 0])
height = 10
V_D = np.array([10, 0, 0])
n = 100
origin_bot = np.array([0, 0, 60])
num_legs = 6
num_servos = 18
cycle_time = 10
num_step = 5
leg_ang = np.pi/2

In [106]:
# Default leg position
def_EF_pos = np.array([195, 0, 0])
print("Def pos")
print(def_EF_pos)

# Inverse kinematics
angles = leg_IK(origin_bot, def_EF_pos, links, 0)
print("IK")
print(np.rad2deg(angles))

# Forward kinematics
pos = leg_FK(origin_bot, angles, links)
print("FK_pos")
print(np.round(pos, 2))

Def pos
[195   0   0]
[195.   -72.75 -60.  ]


ValueError: math domain error